In [16]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import nltk
from collections import defaultdict

# Load dataset
data = pd.read_csv('conversational_data.csv')
print("Dataset loaded:")
print(data.head())

# Define intents
intents = [
    "greeting",
    "get_weather",
    "set_alarm",
    "book_flight",
    "book_hotel",
    "book_taxi",
    "find_restaurant",
    "get_news",
    "schedule_meeting",
    "tell_joke",
    "play_music",
    "make_reservation",
    "get_traffic",
    "get_showtimes",
    "learn_language",
    "order_food",
    "set_reminder",
    "get_quote",
    "find_gym"
]

num_intents = len(intents)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intents)

# Feedback storage
feedback_storage = defaultdict(list)

# Tokenization and fine-tuning (for demo purposes, this is a placeholder)
def classify_intent(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    intent_label = torch.argmax(probs).item()
    return intent_label

# Use a pre-trained NER model for entity extraction
ner_model = pipeline('ner', model="dbmdz/bert-large-cased-finetuned-conll03-english", tokenizer="bert-large-cased")

def process_user_input(text):
    intent_label = classify_intent(text)
    intent = intents[intent_label]
    entities = ner_model(text)
    return intent, entities

def format_output(intent, entities):
    print(f"Intent: {intent}")
    print(f"Entities: {entities}")

# Feedback function
def get_feedback():
    while True:
        try:
            feedback = int(input("Please rate my response (1-5): "))
            if 1 <= feedback <= 5:
                return feedback
            else:
                print("Please enter a valid number between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def adjust_response(intent):
    if feedback_storage[intent]:
        average_feedback = sum(feedback_storage[intent]) / len(feedback_storage[intent])
        if average_feedback <= 2:
            print("I see that my responses for this topic could be improved.")
            print("I'll try to give you better information next time!")
    else:
        print("Thanks for your feedback!")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break

    intent, entities = process_user_input(user_input)

    format_output(intent, entities)

    feedback = get_feedback()
    feedback_storage[intent].append(feedback)
    adjust_response(intent)


Dataset loaded:
                           user_input           intent    entities
0           Book a flight to New York      book_flight    New York
1  What's the weather like in London?      get_weather      London
2                    Order me a pizza       order_food       pizza
3            Find a restaurant nearby  find_restaurant  restaurant
4                     Play some music       play_music         NaN


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on a

You: Book a flight to New York
Intent: get_weather
Entities: [{'entity': 'I-LOC', 'score': 0.9995646, 'index': 5, 'word': 'New', 'start': 17, 'end': 20}, {'entity': 'I-LOC', 'score': 0.9992194, 'index': 6, 'word': 'York', 'start': 21, 'end': 25}]
Please rate my response (1-5): 2
I see that my responses for this topic could be improved.
I'll try to give you better information next time!
You: Book a flight to New York
Intent: get_weather
Entities: [{'entity': 'I-LOC', 'score': 0.9995646, 'index': 5, 'word': 'New', 'start': 17, 'end': 20}, {'entity': 'I-LOC', 'score': 0.9992194, 'index': 6, 'word': 'York', 'start': 21, 'end': 25}]
Please rate my response (1-5): 5
You: exit
Goodbye!
